## Importation des bliblothèques

In [1]:
import sqlite3
import pandas as pd

## Connection à la base de données

In [2]:
connect = sqlite3.connect("../../Databases/raw-database.db")
cursor = connect.cursor()

## Importation des données freeze frames

In [3]:
req = cursor.execute(f"SELECT * FROM freeze_frames")
res = req.fetchall()
desc = req.description
freeze_frames = pd.DataFrame(res)
freeze_frames.columns = [i[0] for i in desc]

In [4]:
# Aperçu des données
freeze_frames.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object
0,4820,2025-01-11 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN
1,4834,2025-01-11 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0
2,4854,2025-01-11 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN


## Importation des données events

In [5]:
# On récupère les event_id SB pour pouvoir importer uniquement ceux qui nous intéressent depuis la table events
event_id_freeze_frames = freeze_frames.event_id.unique().tolist()

In [6]:
req = cursor.execute(f"SELECT event_id, pass_cross, pass_type FROM events WHERE event_id IN ({', '.join('?' * len(event_id_freeze_frames))})",
                     event_id_freeze_frames)
res = req.fetchall()
desc = req.description
events = pd.DataFrame(res)
events.columns = [i[0] for i in desc]

In [7]:
# Aperçu des données
events.head(3)

,event_id,pass_cross,pass_type
0,2e85ad3e-59c4-479a-a478-4c98174f686b,NaN,None
1,872df8f3-a750-4b61-8890-61a35e96b4a4,1.0,None
2,0f842e18-1363-4af3-8faa-6c9cce55b118,NaN,None


## Jointure des events avec les freeze frames

In [8]:
freeze_frames_events = pd.merge(freeze_frames, events, how = "left", on = "event_id")
freeze_frames_events.sort_values(by = ["match_id_SKC", "timestamp"], inplace = True)

In [9]:
# Aperçu des données
freeze_frames_events.head(3)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type
0,4820,2025-01-11 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,away team,NaN,NaN,None
1,4834,2025-01-11 00:04:30.400000,1.0,28239a7e-4d65-4618-99f3-2763088e5e92,42.6125,-26.86,1,1020089,away team,159756.0,1.0,None
2,4854,2025-01-11 00:04:32.400000,1.0,3ea6ecd9-bc39-4ce5-9b47-1eda239f2705,44.1,1.615,1,1020089,away team,NaN,NaN,None


## Analyse des centres

In [10]:
# Création d'un dataframe ne comprenant que les centres du dataframe initial
centres = freeze_frames_events[freeze_frames_events.pass_cross == 1]

In [11]:
# On regarde si il y'a des valeurs centres qui sont présents plusieurs fois
# On utilise l'identifiant ["frame", "match_id_SKC"] qui permet d'identifier, de manière unique, chaque frame des données
centres[["frame", "match_id_SKC"]].duplicated().any()

np.False_

On souhaite maintenant identifier chaque centre et chaque évènement précédant un centre par un centre_id.  
Chaque couple (event précédant le centre, centre) sera alors identifié par un unique centre_id.  

De plus, on part du principe qu'on dispose bien de chaque évènement précédant les centres.  
En effet, étant donné que nous ne nous sommes pas chargé de fusionner les évènements précédant et suivant les centres aux évènements correspondant aux centres, nous ne pouvons pas gérer ces erreurs.

Pour gérer les cas ou l'évènement suivant un centre (centre1) est également un centre (centre2), nous partons du principe que ces centres ne sont pas dupliqués mais plutôt que ces centres se suivent en terme de frame.  
En effet, nous n'aurons pas 2 couples (event avant centre1, centre1, centre2) et (centre1, centre2, event après centre2) mais plutot un couple (event avant centre1, centre1, centre2, event après centre2)

In [12]:
# Nous partons de ce principe car nous n'avons pas de valeur dupliquée pour la colonne event_id :
print(freeze_frames_events.event_id.is_unique)

True


Enfin, il est nécessaire des supprimer les events correspondants à des coups de pied arretés car ce n'est pas cohérent d'étudier les évènements précédant des CPA.

In [13]:
# Liste des CPA définis par Stats Bomb
CPA = ["Corner", "Free Kick", "Goal Kick", "Kick Off"]

# Supression des CPA des centres
centres = centres[~centres.pass_type.isin(CPA)]

In [14]:
# identification des évènements précédant les centres
index_events_avant_centre = centres.index - 1

In [15]:
# Création d'un dataframe ne comprenant que les actions précédents les centres
events_avant_centre = freeze_frames_events.loc[index_events_avant_centre]

In [16]:
# On regarde si il y'a des évènements précédents des centres qui correspondent à des centres
events_avant_centre[events_avant_centre.pass_cross == 1]

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,group,tackable_object,pass_cross,pass_type


In [17]:
# Autre moyen de vérifier
print(len(index_events_avant_centre.intersection(centres.index)))

0


Il n'y en a pas

In [18]:
# Attribution des centre_id à chaque events précédant les centres et à chaque centre
events_avant_centre["centre_id"] = range(len(events_avant_centre))
centres["centre_id"] = range(len(events_avant_centre))

In [19]:
# On attribue aux events précédant les centres l'équipe qui a effectué le centre (suivant ces events)
# En effet, il est possible que l'équipe en possession du ballon lors de l'évènement précédant un centre ne soit pas la même
# que celle qui a effectué le centre en question
# De ce fait, il est nécessaire de remplacer l'équipe actuellement attribuée aux events précédant les centres par les équipes ayant effectué 
# les centres
events_avant_centre.drop("group", axis = 1, inplace = True)
events_avant_centre = pd.merge(events_avant_centre, centres[["group", "centre_id"]], on = "centre_id")

# De plus, pour éviter les confusions, nous allons renomer la colonne "group" par "team_centre"
events_avant_centre.rename({"group" : "team_centre"}, axis = 1, inplace = True)

In [20]:
# Aperçu des données
events_avant_centre.head(2)

,frame,timestamp,period,event_id,event_x,event_y,is_matched,match_id_SKC,tackable_object,pass_cross,pass_type,centre_id,team_centre
0,4820,2025-01-11 00:04:29,1.0,54cbad0c-1f05-4708-9b7f-2cadeae0636b,35.35,-27.71,1,1020089,NaN,NaN,None,0,away team
1,5752,2025-01-11 00:06:02.200000,1.0,c1a7a598-9690-4a32-a733-b7b3b585df83,-43.4,-27.455,1,1020089,NaN,NaN,None,1,None


Nous avons terminé cette première étape qui nous a permis d'analyser et extraire les centres ainsi que les évènements les précédant.  
Nous obtenons finalement un dataframe composé de l'ensemble des events précédant les centres dont nous disposons.  
Ce dataframe contient des informations cruciales telles que la position de l'event ("event_x" et "event_y"), l'identifiant de la frame (le couple ["frame", "match_id_SKC"]) ou encore l'identifiant du centre que nous avons créé, dans le but de pouvoir retrouver le centre auquel correspond l'event précédant un centre.

Nous allons maintenant passer à l'analyse de la position des joueurs au moment de l'event précédant le centre.  
Pour cela, nous allons utiliser les données de tracking que nous avions au préalable importées dans la BDD SQLite grâce à nos programmes d'importation.

# Importation des données de tracking

In [21]:
req = cursor.execute(f"SELECT trackable_object, x, y, frame, match_id_SKC FROM tracking_data")
res = req.fetchall()
desc = req.description
tracking_data = pd.DataFrame(res)
tracking_data.columns = [i[0] for i in desc]

In [22]:
# Aperçu des données
tracking_data.head(2)

,trackable_object,x,y,frame,match_id_SKC
0,12235.0,45.43,-0.97,4820,1020089
1,12327.0,32.26,-2.78,4820,1020089


Nous allons maintenant fusionner le dataframe des events précédant les centres avec le dataframe des données de tracking.  
En effet, on souhaite étudier seulement les positions des joueurs sur les events précédant des centres, or le dataframe tracking_data actuel contient les données de toutes les frames disponibles.  
Pour effectuer ce merge, nous allons joindre les deux dataframes par rapport au couple ["match_id_SKC", "frame"] qui identifie de manière unique chaque frame.

In [23]:
td_avant_centre = pd.merge(events_avant_centre[["match_id_SKC", "frame", "centre_id", "team_centre"]],
                              tracking_data, on = ["match_id_SKC", "frame"])
# td_avant_centre correspond à "tracking data des events avant les centres"

In [24]:
# Aperçu des données
td_avant_centre.head(2)

,match_id_SKC,frame,centre_id,team_centre,trackable_object,x,y
0,1020089,4820,0,away team,12235.0,45.43,-0.97
1,1020089,4820,0,away team,12327.0,32.26,-2.78


Nous allons maintenant regarder s'il y'a des frames (des events précédant les centres) pour lesquelles nous n'avons pas de tracking data

In [25]:
print("Nombre de centres pour lesquels nous n'avons pas d'information sur la frame de l'event précédent le centre dans les données de tracking :",
      len(events_avant_centre[~events_avant_centre.centre_id.isin(td_avant_centre.centre_id)]))
print("Nombre total de frames correspondant à des events précédant un centre :", len(events_avant_centre))

Nombre de centres pour lesquels nous n'avons pas d'information sur la frame de l'event précédent le centre dans les données de tracking : 38
Nombre total de frames correspondant à des events précédant un centre : 6181


On remarque qu'il y'a seulement 38 des 6181 frames pour lesquelles nous n'avons pas de données de tracking, ce qui reste raisonnablement faible.

---

Rappel : nous souhaitons étudier le nombre d'adversaires entre le ballon et le but pendant l'évènement précédant le centre.  
Or, pour l'instant, il nous manque les informations suivantes :
- Les id Skill Corner des équipes (à domicile et extérieur pour chaque match)
- les équipes auxquels appartiennent les joueurs  

De ce fait, nous devons obtenir pour chaque match, les team id Skill Corner des équipes à domicile et à l'extérieure, ainsi que pour chaque joueur, l'équipe à laquelle ils appartiennent.  
C'est ce que nous allons faire dans les deux prochaines sections.  
Il est important de noté que nous ne disposons pas directement de ces informations, nous allons donc devoir manipuler les différentes données de matching (entre les id Stats Bomb et Skill Corner).

# Matching des équipes (domicile et extérieur)

L'objectif de cette partie est de savoir, pour chaque match, quel est l'id Skill Corner de l'équipe à domicile et celui de l'équipe à l'extérieur

Pour obtenir ces informations, nous allons nous servir de des données dont nous disposons actuellement :
- Nous avons les matching des id Skill Corner et Stats Bomb des équipes
- De même pour les id des matches
- Enfin, grâce aux données matches de Stats Bomb (SB_matches), nous savons pour chaque match quelle était l'équipe à domicile et quelle était l'équipe à l'extérieur

En effectuant diverses jointures entre ces tables, nous allons réussir à retrouver les informations souhaitées.

In [26]:
# Importation des données de matching entre les id Skill Corner et Stats Bomb des matches
req = cursor.execute(f"SELECT * FROM matching_matches")
res = req.fetchall()
desc = req.description
matching_matches = pd.DataFrame(res)
matching_matches.columns = [i[0] for i in desc]

In [27]:
# Aperçu des données
matching_matches.head(2)

,match_id_SB,match_id_SKC
0,3894366.0,1547880
1,3894367.0,1547881


In [28]:
# Importation des données de matching entre les id Skill Corner et Stats Bomb des équipes
req = cursor.execute(f"SELECT * FROM matching_teams")
res = req.fetchall()
desc = req.description
matching_teams = pd.DataFrame(res)
matching_teams.columns = [i[0] for i in desc]

In [29]:
# Aperçu des données
matching_teams.head(2)

,team_id_SB,team_id_SKC
0,168,85
1,144,66


In [30]:
# Importation des informations Stats Bomb sur l'ensemble des matches dont nous disposons.
req = cursor.execute(f"SELECT match_id_SB, home_team_id_SB, away_team_id_SB FROM SB_matches")
res = req.fetchall()
desc = req.description
SB_matches = pd.DataFrame(res)
SB_matches.columns = [i[0] for i in desc]

In [31]:
# Aperçu des données
SB_matches.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB
0,3894290,136,139
1,3894367,165,152


Maintenant que nous avons importé l'ensemble des tables (converties en dataframe) nécessaires, nous allons pouvoir effectuer les jointures entre ces dataframes afin de faire matcher les différents id

In [32]:
# Matching des match id SB des données SB_matches avec leur id Skill Corner respectif
merge_match_id = pd.merge(SB_matches, matching_matches, on = "match_id_SB")

In [33]:
# Aperçu des données
merge_match_id.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC
0,3894290,136,139,1434956
1,3894367,165,152,1547881


In [34]:
# Matching des id SB des équipes à domicile des données SB_matches avec leur id Skill Corner respectif
merge_home_team_id = pd.merge(merge_match_id, matching_teams, left_on = "home_team_id_SB", right_on = "team_id_SB")

In [35]:
# Aperçu des données
merge_home_team_id.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,team_id_SB,team_id_SKC
0,3894290,136,139,1434956,136,70
1,3894367,165,152,1547881,165,72


In [36]:
# On retire donc la colonne team_id_SB qui n'est pas utile
merge_home_team_id.drop("team_id_SB", axis = 1, inplace = True)

# De plus, on renomme la colonne "team_id_SKC" par "home_team_id_SKC" car il faut bien spécifier que c'est l'id Skill Corner de l'équipe à domicile
merge_home_team_id.rename({"team_id_SKC" : "home_team_id_SKC"}, axis = 1, inplace = True)


In [37]:
# Il ne nous reste plus qu'à faire la même chose pour les équipes à l'extérieur
matching_home_away_team = pd.merge(merge_home_team_id, matching_teams, left_on = "away_team_id_SB", right_on = "team_id_SB")
matching_home_away_team.drop("team_id_SB", axis = 1, inplace = True)
matching_home_away_team.rename({"team_id_SKC" : "away_team_id_SKC"}, axis = 1, inplace = True)

In [38]:
# Aperçu des données
matching_home_away_team.head(2)

,match_id_SB,home_team_id_SB,away_team_id_SB,match_id_SKC,home_team_id_SKC,away_team_id_SKC
0,3894290,136,139,1434956,70,86
1,3894367,165,152,1547881,72,328


Nous ne souhaitons conserver que les informations qui nous serons utiles pour le matching avec les données de tracking, c'est à dire les colonnes  "match_id_SKC", "home_team_id_SKC" et "away_team_id_SKC".

In [39]:
matching_home_away_team = matching_home_away_team[["match_id_SKC", "home_team_id_SKC", "away_team_id_SKC"]]

# Matching des joueurs avec leur équipe (pour chaque match)

Nous allons maintenant faire matcher, pour chaque match, l'ensemble des id Stats Bomb des joueurs présents sur la feuille de match avec leur id Skill Corner respectif.  
Il est important de bien s'intéresser aux id des joueurs pour chaque match et non sur une saison.  
En effet, dans le cas ou un joueur serait transféré d'une équipe à une autre au cours d'une saison, il aurait alors 2 valeurs (dans le cas ou on ferait matcher le joueur avec son équipe sur la saison)

Pour obtenir ce matching, nous allons utiliser les données lineup de Stats Bomb qui nous fournissent, pour chaque match, la liste des id des joueurs sur la feuille de match ainsi que les id SB des équipes auxquelles ils appartiennent.  
De plus, nous utiliserons les tables de matching pour les id SB et SK des matches, des équipes et des joueurs.

In [40]:
# Importation des données lineup de Stats Bomb.
req = cursor.execute(f"SELECT player_id_SB, team_id_SB, match_id_SB FROM lineups")
res = req.fetchall()
desc = req.description
lineups = pd.DataFrame(res)
lineups.columns = [i[0] for i in desc]

In [41]:
# Aperçu des données
lineups.head(2)

,player_id_SB,team_id_SB,match_id_SB
0,2941,147,3894037
1,3044,147,3894037


In [42]:
# Importation des données de matching des id SK et SB des joueurs.
# De plus, nous allons aussi importer la colonne "trackable_object" qui est utilisée pour identifier les joueurs dans les données de tracking
req = cursor.execute(f"SELECT trackable_object, player_id_SB, player_id_SKC FROM matching_players")
res = req.fetchall()
desc = req.description
matching_players = pd.DataFrame(res)
matching_players.columns = [i[0] for i in desc]

In [43]:
# Aperçu des données
matching_players.head(2)

,trackable_object,player_id_SB,player_id_SKC
0,820104,307251.0,818541
1,39010,66886.0,37889


Nous allons maintenant faire matcher les id SB des joueurs, des équipes et des matches avec leur id SK respectif

In [44]:
# Matching des id des matches
# Nous nous permettons de réutiliser le nom "merge_match_id" pour le dataframe, que nous avons déjà utilisé précédemment.
# En effet, ce sont des dataframes temporaires que nous ne réutiliserons pas donc on n'a pas besoin de garder leur contenu.
merge_match_id = pd.merge(lineups, matching_matches, on = "match_id_SB")

In [45]:
# Aperçu des données
merge_match_id.head(2)

,player_id_SB,team_id_SB,match_id_SB,match_id_SKC
0,2941,147,3894037,1020753
1,3044,147,3894037,1020753


In [46]:
# Matching des id des joueurs
merge_player_id = pd.merge(merge_match_id, matching_players, on = "player_id_SB")

In [47]:
# Aperçu des données
merge_player_id.head(2)

,player_id_SB,team_id_SB,match_id_SB,match_id_SKC,trackable_object,player_id_SKC
0,2941,147,3894037,1020753,11308,11274
1,3044,147,3894037,1020753,1743,1733


In [48]:
# Enfin, il ne nous reste plus qu'à faire matcher les id des équipes
matching_player_team = pd.merge(merge_player_id, matching_teams, on = "team_id_SB")

In [49]:
# Aperçu des données
matching_player_team.head(2)

,player_id_SB,team_id_SB,match_id_SB,match_id_SKC,trackable_object,player_id_SKC,team_id_SKC
0,2941,147,3894037,1020753,11308,11274,80
1,3044,147,3894037,1020753,1743,1733,80


Nous ne souhaitons conserver que les informations qui nous serons utiles pour le matching avec les données de tracking, c'est à dire les colonnes  "match_id_SKC", "trackable_object" et "team_id_SKC".

In [50]:
matching_player_team = matching_player_team[["match_id_SKC", "trackable_object", "team_id_SKC"]]

---

Nous avons maintenant terminé de créer les données de matching nécessaire, nous allons donc les appliquer aux données de tracking.

In [51]:
# Rappel données de tracking
td_avant_centre.head(2)

,match_id_SKC,frame,centre_id,team_centre,trackable_object,x,y
0,1020089,4820,0,away team,12235.0,45.43,-0.97
1,1020089,4820,0,away team,12327.0,32.26,-2.78


Dans un premier temps, nous allons faire matcher les joueurs avec leur équipe

In [52]:
# Nous rajoutons le paramètre : how = "left" afin d'effectuer un "left join" SQL.
# En effet, les données de tracking contiennent aussi des informations sur le ballon, qui ne possède pas d'équipe.
# En effectuant un inner join (qui est la jointure effectuée par défaut par la commande merge de Pandas), cela ne gardera pas les données de
# tracking du ballon car il n'est pas référencé dans les données de matching_player_team
matching_player_tdac = pd.merge(td_avant_centre, matching_player_team, on = ["match_id_SKC", "trackable_object"], how = "left")
# tdac (de matching_player_tdac) signifie "tracking data avant centre"

In [53]:
# Aperçu des données
matching_player_tdac.head(2)

,match_id_SKC,frame,centre_id,team_centre,trackable_object,x,y,team_id_SKC
0,1020089,4820,0,away team,12235.0,45.43,-0.97,70.0
1,1020089,4820,0,away team,12327.0,32.26,-2.78,70.0


In [54]:
# Rappel des données de matching des équipes (à domicile et extérieur) pour chaque match
matching_home_away_team.head(2)

,match_id_SKC,home_team_id_SKC,away_team_id_SKC
0,1434956,70,86
1,1547881,72,328


Nous allons maintenant effectuer le matching des id des équipes à domicile et extérieur.  
Cependant, nous souhaiterions effectuer un merge au niveau des colonnes "match_id_SKC" et "team_centre", ce qui n'est pas possible actuellement au vu du format des données de matching des équipes  
Pour cela, nous allons créer deux nouveaus dataframes "matching_home_team" et "matching_away_team".

In [55]:
# Création des deux nouveaux dataframes
matching_home_team = matching_home_away_team[["match_id_SKC", "home_team_id_SKC"]]
matching_away_team = matching_home_away_team[["match_id_SKC", "away_team_id_SKC"]]

In [56]:
# Création des colonnes "team_centre" à ces deux dataframes qui seront utilisées pour la jointure
matching_home_team["team_centre"] = "home team"
matching_away_team["team_centre"] = "away team"

In [57]:
# Aperçu des données
matching_home_team.head(1)

,match_id_SKC,home_team_id_SKC,team_centre
0,1434956,70,home team


In [58]:
# Aperçu des données
matching_away_team.head(1)

,match_id_SKC,away_team_id_SKC,team_centre
0,1434956,86,away team


In [59]:
# Jointure de ces deux dataframes au dataframe des données de tracking
# Le paramètre : how = "left" nous permettra de vérifier si on a bien l'information des équipes à domicile
# pour l'ensemble des matches dont nous étudions les tracking data
# En effet, si on a une valeur Null, cela voudra dire qu'il nous manque l'information
matching_player_home_team_tdac = pd.merge(matching_player_tdac, matching_home_team, how = "left", on = ["match_id_SKC", "team_centre"])

# De même pour les équipes à l'extérieur
matching_player_home_away_team_tdac = pd.merge(matching_player_home_team_tdac, matching_away_team, how = "left",
                                               on = ["match_id_SKC", "team_centre"])

In [61]:
matching_player_home_away_team_tdac.head(24)

,match_id_SKC,frame,centre_id,team_centre,trackable_object,x,y,team_id_SKC,home_team_id_SKC,away_team_id_SKC
0,1020089,4820,0,away team,12235.0,45.43,-0.97,70.0,NaN,78.0
1,1020089,4820,0,away team,12327.0,32.26,-2.78,70.0,NaN,78.0
2,1020089,4820,0,away team,2340.0,33.55,-16.15,70.0,NaN,78.0
3,1020089,4820,0,away team,12788.0,30.18,5.24,70.0,NaN,78.0
4,1020089,4820,0,away team,25539.0,30.43,-24.45,70.0,NaN,78.0
5,1020089,4820,0,away team,28826.0,25.73,-17.81,70.0,NaN,78.0
6,1020089,4820,0,away team,20395.0,13.19,-18.72,70.0,NaN,78.0
7,1020089,4820,0,away team,7771.0,19.47,-12.85,70.0,NaN,78.0
8,1020089,4820,0,away team,11721.0,6.75,2.53,70.0,NaN,78.0
9,1020089,4820,0,away team,4930.0,18.14,-24.22,70.0,NaN,78.0
